# **Pandas - Basic (2)**

기본적으로 Series와 DataFrame을 만드는 법을 줄 살펴본 지난 포스팅 Pandas - Basic (1)에 이어 이번에는 Basic (2)편을 준비해보았다.
우선 실제로 데이터를 다룰 때 많이 사용하게 되는 groupby, select, merge에 대해 정리해보고, 이어서 csv나 excel 파일로부터 데이터를 불러오고 내보내는 방법에 대해 다루어보자.

## 1. **groupby**
- DataFrame에서 특정 column의 unique 값만 가지고 새로운 DataFrame을 생성 
- size 방법: `dfname.groupby("column_name").size()`
- agg 방법: `dfname.groupby("column_name").agg('min'|'max'|'mean' 등)`

In [1]:
import pandas as pd

In [3]:
# 사람 이름과 나이가 들어간 데이터를 만들기
import random, string

def get_name():
    names = ['Adam', 'Alan', 'Alex', 'Alvin', 'Andrew', 'Anthony', 'Arnold', 'Jim', 'Baldy','Peter']
    return random.choice(names)


def get_age(start=20, end=40):
    return random.randint(start, end)    # random 모듈의 randint는 end가 포함됨


def make_data(rows=10):     # default로 데이터 10개를 설정함
    datas = []
    for _ in range(rows):   # '_"를 쓰는 이유는 이름 중복 피하려고(컨벤션) 
        data = {"Age":get_age(), "Name":get_name()}
        datas.append(data)
    return datas

In [4]:
# make DataFrame
g_df = pd.DataFrame(make_data(20))
g_df

,Age,Name
0,21,Alvin
1,39,Alan
2,27,Baldy
3,34,Anthony
4,33,Arnold
5,30,Alex
6,23,Andrew
7,38,Anthony
8,30,Andrew
9,26,Peter


In [11]:
# unique name list 1: set을 이용한 형변환으로 유니크 이름 추출
result1 = np.array(list(set(g_df["Name"].values)))
print(len(result1), result1)

9 ['Baldy' 'Peter' 'Adam' 'Alan' 'Anthony' 'Alex' 'Andrew' 'Alvin' 'Arnold']


In [12]:
# unique name list 2: unique 함수를 이용하여 유니크 이름을 뽑는다
result2 = list(g_df["Name"].unique())
print(len(result2), result2)

9 ['Alvin', 'Alan', 'Baldy', 'Anthony', 'Arnold', 'Alex', 'Andrew', 'Peter', 'Adam']


### 1.1 groupby - size
- 각 이름별이 몇 번 나왔는지 counts 컬럼이 추가된 데이터프레임 만들기
- syntax: `dfname.groupby("column_name").size()`
    - data가 몇 번 나오는지 count
- Series로 출력됨

In [8]:
result_df = g_df.groupby("Name").size()
result_df        

Name
Adam       1
Alan       1
Alex       3
Alvin      2
Andrew     5
Anthony    2
Arnold     3
Baldy      1
Peter      2
dtype: int64

In [9]:
# counts라는 컬럼 이름으로 index를 리셋
result_df = g_df.groupby("Name").size().reset_index(name='counts')
result_df

,Name,counts
0,Adam,1
1,Alan,1
2,Alex,3
3,Alvin,2
4,Andrew,5
5,Anthony,2
6,Arnold,3
7,Baldy,1
8,Peter,2


In [10]:
# sort_values를 이용하여 counts로 내림차순으로 정렬한다.
result_df = result_df.sort_values(by=['counts'], ascending=False)

# 인덱스를 다시 리셋한다.
result_df.reset_index(drop=True, inplace=True)
result_df

,Name,counts
0,Andrew,5
1,Alex,3
2,Arnold,3
3,Alvin,2
4,Anthony,2
5,Peter,2
6,Adam,1
7,Alan,1
8,Baldy,1


### 1.2 groupby - agg
- 각 이름별 age의 최소값/최대값/평균 등을 나타내는 column을 나타내기
- syntax: `dfname.groupby("column_name").agg('min'|'max'|'mean' 등)`

In [13]:
# agg: min
g_df.groupby("Name").agg('min').reset_index()

,Name,Age
0,Adam,34
1,Alan,39
2,Alex,30
3,Alvin,21
4,Andrew,23
5,Anthony,34
6,Arnold,23
7,Baldy,27
8,Peter,26


In [14]:
# agg: max
g_df.groupby("Name").agg('max').reset_index()

,Name,Age
0,Adam,34
1,Alan,39
2,Alex,35
3,Alvin,39
4,Andrew,30
5,Anthony,38
6,Arnold,40
7,Baldy,27
8,Peter,33


In [15]:
# agg : mean
g_df.groupby("Name").agg('mean').reset_index()

,Name,Age
0,Adam,34.000000
1,Alan,39.000000
2,Alex,32.333333
3,Alvin,30.000000
4,Andrew,27.400000
5,Anthony,36.000000
6,Arnold,32.000000
7,Baldy,27.000000
8,Peter,29.500000


In [16]:
# 여러가지 함수를 agg할 수 있음
df = g_df.groupby("Name").agg(["min","max","mean"]).reset_index()
df

Name Age               
           min max       mean
0     Adam  34  34  34.000000
1     Alan  39  39  39.000000
2     Alex  30  35  32.333333
3    Alvin  21  39  30.000000
4   Andrew  23  30  27.400000
5  Anthony  34  38  36.000000
6   Arnold  23  40  32.000000
7    Baldy  27  27  27.000000
8    Peter  26  33  29.500000

## **2. select**

### 2.1 head, tail
- head: dataframe 상단 n개 row를 보여줌 (default 5줄)
    - syntax: `df.head(n=5)`
- tail: dataframe 하단 n개 row를 보여줌 (default 5줄)
    - syntax: `df.tail(n=5)`

In [17]:
# 위에 5개 출력
df.head()

# # 위에 3개 출력
# df.head(3)

Name Age               
          min max       mean
0    Adam  34  34  34.000000
1    Alan  39  39  39.000000
2    Alex  30  35  32.333333
3   Alvin  21  39  30.000000
4  Andrew  23  30  27.400000

In [18]:
# 아래에 5개 출력: tail을 사용하면 index를 보고 데이터가 몇개인지도 함께 확인 가능
df.tail()

# # 아래에 3개 출력
# df.tail(3)

Name Age          
           min max  mean
4   Andrew  23  30  27.4
5  Anthony  34  38  36.0
6   Arnold  23  40  32.0
7    Baldy  27  27  27.0
8    Peter  26  33  29.5

### 2.2 indexing, slicing

In [19]:
# offset index - 3~5줄 데이터 출력 
df[3:6]

Name Age          
           min max  mean
3    Alvin  21  39  30.0
4   Andrew  23  30  27.4
5  Anthony  34  38  36.0

In [20]:
# 3~끝 데이터 출력
df[3:]

Name Age          
           min max  mean
3    Alvin  21  39  30.0
4   Andrew  23  30  27.4
5  Anthony  34  38  36.0
6   Arnold  23  40  32.0
7    Baldy  27  27  27.0
8    Peter  26  33  29.5

In [21]:
# 시작~3 데이터 출력
df[:4]

Name Age               
         min max       mean
0   Adam  34  34  34.000000
1   Alan  39  39  39.000000
2   Alex  30  35  32.333333
3  Alvin  21  39  30.000000

In [22]:
# 데이터 거꾸로 출력
df[::-1]

Name Age               
           min max       mean
8    Peter  26  33  29.500000
7    Baldy  27  27  27.000000
6   Arnold  23  40  32.000000
5  Anthony  34  38  36.000000
4   Andrew  23  30  27.400000
3    Alvin  21  39  30.000000
2     Alex  30  35  32.333333
1     Alan  39  39  39.000000
0     Adam  34  34  34.000000

In [26]:
# 범위는 offset이 가능하지만 df[3] → error, `loc`을 써야 함
df.loc[3]

Name          Alvin
Age   min        21
      max        39
      mean       30
Name: 3, dtype: object

### 2.3 index, column으로 데이터 호출
- `df[index][column]`으로 자료 호출 (two-level column도 가능)

In [28]:
# 2번 데이터 나이 최소값, 3번 데이터 이름
print(df.loc[2]["Age"]["min"])
print(df.loc[3]["Name"][""])

30
Alvin


In [29]:
# two-level column
df["Age"]["min"]

0    34
1    39
2    30
3    21
4    23
5    34
6    23
7    27
8    26
Name: min, dtype: int64

In [30]:
df["Age"]["min"][3]     

21

In [31]:
# 2-레벨 column을 1-level로 바꾸기
data = {"Name":df["Name"],
        "Min":df["Age"]["min"],
        "Max":df["Age"]["max"],
        "Mean":df["Age"]["mean"]}

n_df = pd.DataFrame(data)
n_df

,Name,Min,Max,Mean
0,Adam,34,34,34.000000
1,Alan,39,39,39.000000
2,Alex,30,35,32.333333
3,Alvin,21,39,30.000000
4,Andrew,23,30,27.400000
5,Anthony,34,38,36.000000
6,Arnold,23,40,32.000000
7,Baldy,27,27,27.000000
8,Peter,26,33,29.500000


평균 나이가 30살 이상인 데이터를 Mean values로 내림차수으로 정렬하고 인덱스를 재설정하기

In [32]:
n_df[n_df["Mean"] > 30].sort_values(by=["Mean", "Max"], ascending=False).reset_index(drop=True)

,Name,Min,Max,Mean
0,Alan,39,39,39.000000
1,Anthony,34,38,36.000000
2,Adam,34,34,34.000000
3,Alex,30,35,32.333333
4,Arnold,23,40,32.000000


In [33]:
# 각 이름 count 컬럼 추가
n_df["Count"] = list(g_df.groupby("Name").size())
n_df

,Name,Min,Max,Mean,Count
0,Adam,34,34,34.000000,1
1,Alan,39,39,39.000000,1
2,Alex,30,35,32.333333,3
3,Alvin,21,39,30.000000,2
4,Andrew,23,30,27.400000,5
5,Anthony,34,38,36.000000,2
6,Arnold,23,40,32.000000,3
7,Baldy,27,27,27.000000,1
8,Peter,26,33,29.500000,2


In [34]:
# Mean 컬럼을 맨 끝으로 이동하기 
mean = n_df["Mean"]                          # Mean 데이터를 Series로 저장
n_df.drop('Mean', axis=1, inplace=True)      # drop: Mean 데이터를 삭제
n_df["Mean"] = mean
n_df

,Name,Min,Max,Count,Mean
0,Adam,34,34,1,34.000000
1,Alan,39,39,1,39.000000
2,Alex,30,35,3,32.333333
3,Alvin,21,39,2,30.000000
4,Andrew,23,30,5,27.400000
5,Anthony,34,38,2,36.000000
6,Arnold,23,40,3,32.000000
7,Baldy,27,27,1,27.000000
8,Peter,26,33,2,29.500000


In [35]:
# rename column
n_df.rename(columns={"Name":"Unique_Name", "Max":"MAX"}, inplace=True)
n_df

,Unique_Name,Min,MAX,Count,Mean
0,Adam,34,34,1,34.000000
1,Alan,39,39,1,39.000000
2,Alex,30,35,3,32.333333
3,Alvin,21,39,2,30.000000
4,Andrew,23,30,5,27.400000
5,Anthony,34,38,2,36.000000
6,Arnold,23,40,3,32.000000
7,Baldy,27,27,1,27.000000
8,Peter,26,33,2,29.500000


## **3. merge**
- merge란 sql의 join과 같은 개념
- 두 개의 데이터 프레임을 하나로 합쳐서 데이터를 보여줄 수 있음

그럼, 아래와 같이 두 개의 데이터프레임을 만들어서 merge를 해보자.
- user_df: 아이디, 이름, 나이 데이터 프레임 생성
- money_df: 아이디, 돈으로 데이터 프레임 생성

user_df 만들기

In [36]:
# user_df: 중복되는 이름 없이 아이디, 이름, 나이 데이터가 포함된 데이터 프레임 생성
user_df = pd.DataFrame(columns=["UserID", "Name", "Age"])

for idx in range(1,9):    # 무한루프에 빠지지 않도록 위에서 넣어준 이름보다 적게 넣기
    name = get_name()

    # 이름이 중복이면 다시 뽑기
    while name in list(user_df["Name"]):
        name = get_name()
    # 데이터 name_df에 insert
    data = {"Name":name, "UserID":idx, "Age":get_age()}
    user_df.loc[len(user_df)] = data    # 마지막 줄에 데이터 추가

    
user_df

,UserID,Name,Age
0,1,Baldy,32
1,2,Alan,34
2,3,Adam,27
3,4,Andrew,36
4,5,Peter,22
5,6,Arnold,36
6,7,Jim,28
7,8,Alvin,24


money_df 만들기

In [37]:
# money_df: ID와 Money 데이터가 있는 데이터 프레임 생성
money_df = pd.DataFrame(columns=["ID","Money"])

for idx in range(15):
    money = random.randint(1,20) * 1000
    data = {"Money":money, "ID":random.randint(1,9)}
    money_df.loc[len(money_df)] = data


# money_df.sort_values("ID", inplace=True)
money_df

,ID,Money
0,6,16000
1,9,3000
2,7,3000
3,2,6000
4,7,11000
5,6,4000
6,9,18000
7,9,15000
8,5,10000
9,6,6000


money_df의 ID 컬럼과 user_df의 UserID(key값)를 매칭 시켜 merge

In [38]:
# UserID와 ID 데이터는 같지만 컬럼명이 다르기 때문에 두개의 컬럼 모두 데이터 프레임에 들어감
result_df = money_df.merge(user_df, left_on="ID", right_on="UserID", how='outer')
result_df

,ID,Money,UserID,Name,Age
0,6,16000,6,Arnold,36
1,6,4000,6,Arnold,36
2,6,6000,6,Arnold,36
3,9,3000,NaN,NaN,NaN
4,9,18000,NaN,NaN,NaN
5,9,15000,NaN,NaN,NaN
6,7,3000,7,Jim,28
7,7,11000,7,Jim,28
8,7,20000,7,Jim,28
9,2,6000,2,Alan,34


In [39]:
# 컬럼명을 동일하게 변경한후에 merge
# left_on과 right_on을 적어줄 필요가 없다.
user_df.rename(columns={"UserID":"ID"}, inplace=True)
result_df = pd.merge(money_df, user_df)
result_df

,ID,Money,Name,Age
0,6,16000,Arnold,36
1,6,4000,Arnold,36
2,6,6000,Arnold,36
3,7,3000,Jim,28
4,7,11000,Jim,28
5,7,20000,Jim,28
6,2,6000,Alan,34
7,2,10000,Alan,34
8,5,10000,Peter,22
9,4,12000,Andrew,36


In [40]:
# 각 이름으로 groubpy하고 Money 데이터를 모두 sum한 결과의 인덱스를 리셋한다.
money_list = result_df.groupby("Name").sum()["Money"].reset_index()

# Money데이터를 내림차순으로 정렬하고 index를 리셋한다.
money_list = money_list.sort_values(by=['Money'], ascending=False).reset_index(drop=True)
money_list

,Name,Money
0,Jim,34000
1,Arnold,26000
2,Alan,16000
3,Alvin,14000
4,Andrew,12000
5,Peter,10000
6,Baldy,5000


In [42]:
# how에 outer를 사용하면 데이터가 없는 사람은 Money가 NaN으로 출력된다.
# fillna(value=0) : NaN 데이터를 0으로 채운다.
result = pd.merge(user_df, money_list, how='outer').fillna(value=0)
result = result.sort_values(by=['Money'], ascending=False).reset_index(drop=True)
result

,ID,Name,Age,Money
0,7,Jim,28,34000.0
1,6,Arnold,36,26000.0
2,2,Alan,34,16000.0
3,8,Alvin,24,14000.0
4,4,Andrew,36,12000.0
5,5,Peter,22,10000.0
6,1,Baldy,32,5000.0
7,3,Adam,27,0.0


In [45]:
# column의 형변환
result["Money"] = result["Money"].astype("int")
result

,ID,Name,Age,Money
0,7,Jim,28,34000
1,6,Arnold,36,26000
2,2,Alan,34,16000
3,8,Alvin,24,14000
4,4,Andrew,36,12000
5,5,Peter,22,10000
6,1,Baldy,32,5000
7,3,Adam,27,0


## **4. input / output**
- csv 파일: 쉼표`(,)`로 값을 구분하는 파일 타입
- 엑셀 파일: 
    - 인코딩 타입이 UTF-8이 아님
    - 영어가 아닌 다른 언어를 저장하거나 로드할 때 인코딩을 주의해야 함

In [46]:
result

,ID,Name,Age,Money
0,7,Jim,28,34000
1,6,Arnold,36,26000
2,2,Alan,34,16000
3,8,Alvin,24,14000
4,4,Andrew,36,12000
5,5,Peter,22,10000
6,1,Baldy,32,5000
7,3,Adam,27,0


### 4.1 csv 파일 읽고 내보내기

#### (1) csv 파일로 DataFrame 내보내기

In [47]:
result.to_csv("foo.csv", index=False)

#### (2) csv 파일 DataFrame으로 불러오기

In [48]:
pd.read_csv("foo.csv")    #index가 자동으로 붙음 (그래서 저장할 때 False로 함)

,ID,Name,Age,Money
0,7,Jim,28,34000
1,6,Arnold,36,26000
2,2,Alan,34,16000
3,8,Alvin,24,14000
4,4,Andrew,36,12000
5,5,Peter,22,10000
6,1,Baldy,32,5000
7,3,Adam,27,0


### 4.2 excel 파일 읽고 내보내기
- 우선 다음의 패키지를 설치한다.
- `$ pip install xlrd` 
- `$ pip install openpyxl`

In [57]:
import openpyxl
import xlrd

#### (1) xlsx 파일로 내보내기

In [53]:
result.to_excel("foo.xlsx", sheet_name="1st_sheet")

#### (2) xlsx 파일 DataFrame으로 읽어오기

In [58]:
pd.read_excel("foo.xlsx","1st_sheet")

,Unnamed: 0,ID,Name,Age,Money
0,0,7,Jim,28,34000
1,1,6,Arnold,36,26000
2,2,2,Alan,34,16000
3,3,8,Alvin,24,14000
4,4,4,Andrew,36,12000
5,5,5,Peter,22,10000
6,6,1,Baldy,32,5000
7,7,3,Adam,27,0


#### 참고자료
- 패스트캠퍼스, ⟪데이터사이언스스쿨 8기⟫ 수업자료